- List of Moves in order of time
- List of Moves based on column (may need to remove Moves when it's measured and no longer relevant to that column) (stored as Index)
- class Move
  - Index (location in the list of Moves)
  - Column_1
  - Column_2
  - Prob_1
  - Prob_2
  - Player
- Board
  5 rows x 5 columns
- Display for board?
- Measure column
  - Iterate over Moves in the column 
    - If Move has Prob_1 less than maxProb
      - Get 0 or 1 from qc
      - Change to: Prob_1=maxProb, Prob_2=0, Column_1=measurement result, Column_2=doesn't matter
- Game logic
  - Is a move valid (# of Moves in that )
  - Interaction with player
- List to say which columns are deterministically full
- Check for game end (when all columns are full)
  - Simulation of classical connect 4

- Players can play a piece into a superposition of two columns. For example, 1/2 C1 + 1/2 C2 is a move that has a 50% chance of landing in column 1 and a 50% chance of landing in column 2.
- A column is measured when a player wants to play into a potentially filled column. If the column is measured to have space, the player's piece will be played into that column. Otherwise, no piece will be played and it becomes the next player's turn.
- A player wins if they get a row, column, or diagonal of four of their adjacent pieces.
- The game ends when a player wins or if no more moves can be made.

In [4]:
NUM_COLUMN = 5
NUM_ROW = 5
MAX_PROB = 8
# Initialize game state
movesByTime = []
movesByColumn = [[] for i in range(NUM_COLUMN)]
fullColumns = [0 for i in range(NUM_COLUMN)]
time = 0
curPlayer = 0

In [ ]:
class Move:
    def __init__(self, i, c1, c2, p1, p2, pl):
        self.index = i
        # Ensure Column 1 is "dominant" in probability; in particular, ensure if
        # a probability is 0 then it's Prob 2
        if p1 < p2:
            c1, c2 = c2, c1
            p1, p2 = p2, p1
        self.col1 = c1
        self.col2 = c2
        self.prob1 = p1
        self.prob2 = p2
        self.player = pl

RNG SYSTEM THAT WILL USE THE QUANTUM COMPUTER

In [ ]:
def _get_bits_from_counts(counts):
    for k, v in counts.items():
      if v ==1:
          return k

In [ ]:
def _request_bits(n):
  global _bitCache
  global _circuit
  iterations = math.ceil(n * 2/_circuit.width())
  for _ in range(iterations):
    job = _backend.run(_circuit, shots=1)
    _bitCache += _get_bits_from_counts(job.result().get_counts())

In [ ]:
def _get_bit_string(n):
  global _bitCache
  if len(_bitCache) < n:
    _request_bits(n-len(_bitCache)) #request as many bits as we are lacking
  bitString = _bitCache[0:n] #result is the first n bits of the bitCache
  _bitCache = _bitCache[n:] #delete the first n bits of the bitCache
  return bitString

In [ ]:
def get_collapsed_piece(n): # there is a n/8 chance that the piece collapses to left
  n = n % 8
  measure = _get_bit_string(3)
  return measure >= n # 0 is left and 1 is right

In [ ]:
def measure(col):
  # output: 
  for t in movesByColumn[col]:
    move = movesByTime[t]
    # If the move is not already collapsed
    if move.prob2 != 0:
      # Get what the piece collapses to
      whichMove = get_collapsed_piece(move.prob1)
      # If the piece collapsed to column 2
      if whichMove == 1:
        # Rename so the piece collapsed to "column 1" (i.e. swap columns 1 and 2)
        move.col1, move.col2 = move.col2, move.col1
      # Either way, one probability is 1 and the other is 0
      move.prob1, move.prob2 = MAX_PROB, 0
      # This move no longer affect column 2
      movesByColumn[move.col2].remove(t)
  # At this point, all moves in this column are collapsed, so if the columns is still full, mark so!
  if len(movesByColumn[col]==NUM_ROW):
    fullColumns[col]=1

In [ ]:
# def getMove(pl):
#   global time
#   # Display valid move inputs (which columns are valid? How to input probabilities? What to do if they don't want to use a superposition?)

#   legalMoves = []
#   for i in range(NUM_COLUMN):
#     if i not in fullColumns:
#       legalMoves.append(str(i))

#   # Prompt for primary column choice
#   print("Possible moves:\n")
#   print(legalMoves)
#   print("Choose a column:\n")
#   c1 = input().strip()
#   while c1 not in legalMoves:
#     print("Invalid input. Try again.\n")
#     print("Possible moves:\n")
#     print(legalMoves)
#     print("Choose a column:\n")
#     c1 = input().strip()
#   legalMoves.remove(c1)
#   c1 = int(c1)

#   # prompt for option to superimpose another column
#   print("Do you want to superimpose another column? (Y/N):\n")
#   opt = input().strip().upper()
#   while (opt != 'Y') and (opt != 'N'):
#     print("Invalid input. Try again.\n")
#     print("Do you want to superimpose another column? (Y/N):\n")
#     opt = input().strip().upper()

#   if opt == 'Y':
#     # prompt for secondary column choice
#     print("Possible moves:\n")
#     print(legalMoves)
#     print("Choose a second column:\n")
#     c2 = input().strip()
#     while c2 not in legalMoves:
#       print("Invalid input. Try again.\n")
#       print("Possible moves:\n")
#       print(legalMoves)
#       print("Choose a second column:\n")
#       c2 = input().strip()
#     c2 = int(c2)

#     # prompts probability
#     options = range(1, MAX_PROB)
#     legalProbs = ""
#     for i in options:
#       legalProbs = legalProbs + f"({i}): {i}/{MAX_PROB}\n"
#     print("Enter a probability for the second column:\n")
#     print(f"Choose one: \n{legalProbs} \n")
#     p2 = int(input().strip())
#     while p2 not in options:
#       print("Invalid input. Try again.\n")
#       print("Enter a probability for the second column:\n")
#       print(f"Choose one: {legalProbs} \n")
#       p2 = int(input().strip())
#     p1 = 8 - p2
#   else:
#     # when no superposition occurs
#     c2 = c1
#     p1 = MAX_PROB
#     p2 = 0

#   newMove = Move(time, c1, c2, p1, 1-p1, pl)
#   movesByTime.append(newMove)
#   movesByColumn[c1].append(newMove.index)
#   if newMove.prob2 > 0:
#     movesByColumn[c2].append(newMove.index)
#   time+=1

In [ ]:
def getMove(pl):
  global time
  # Display valid move inputs (which columns are valid? How to input probabilities? What to do if they don't want to use a superposition?)

  legalMoves = []
  for i in range(NUM_COLUMN):
    if i not in fullColumns:
      legalMoves.append(str(i))

  # Prompt for primary column choice
  print("Possible moves:\n")
  print(legalMoves)
  print("Choose a column:\n")
  c1 = input().strip()
  while c1 not in legalMoves:
    print("Invalid input. Try again.\n")
    print("Possible moves:\n")
    print(legalMoves)
    print("Choose a column:\n")
    c1 = input().strip()
  legalMoves.remove(c1)
  c1 = int(c1)

  # prompt for option to superimpose another column
  print("Do you want to superimpose another column? (Y/N):\n")
  opt = input().strip().upper()
  while (opt != 'Y') and (opt != 'N'):
    print("Invalid input. Try again.\n")
    print("Do you want to superimpose another column? (Y/N):\n")
    opt = input().strip().upper()

  if opt == 'Y':
    # prompt for secondary column choice
    print("Possible moves:\n")
    print(legalMoves)
    print("Choose a second column:\n")
    c2 = input().strip()
    while c2 not in legalMoves:
      print("Invalid input. Try again.\n")
      print("Possible moves:\n")
      print(legalMoves)
      print("Choose a second column:\n")
      c2 = input().strip()
    c2 = int(c2)

    # prompts probability
    options = range(1, MAX_PROB)
    legalProbs = ""
    for i in options:
      legalProbs = legalProbs + f"({i}): {i}/{MAX_PROB}\n"
    print("Enter a probability for the second column:\n")
    print(f"Choose one: \n{legalProbs} \n")
    p2 = int(input().strip())
    while p2 not in options:
      print("Invalid input. Try again.\n")
      print("Enter a probability for the second column:\n")
      print(f"Choose one: {legalProbs} \n")
      p2 = int(input().strip())
  else:
    # when no superposition occurs
    c2 = c1
    p1 = MAX_PROB
    p2 = 0



  # Update move lists and time
  if c1 == c2:
    p1 = MAX_PROB
  newMove = Move(time, c1, c2, p1, 1-p1, pl)
  movesByTime.append(newMove)
  movesByColumn[c1].append(newMove.index)
  if newMove.prob2 > 0:
    movesByColumn[c2].append(newMove.index)
  time+=1

In [ ]:
def checkMeasure(col):
  if len(movesByColumn[col])==NUM_ROW:
    print("Column " + col + " may be full, measuring...")
    measure(col)
    print("After measuring, here is the board:")
    displayBoard()
    if fullColumns[col]:
      print("Column " + col + " is full! Please submit a new move.")
      return False
  return True

In [ ]:
def checkValid(col1, col2, p1):
  if p1 > MAX_PROB or p1 < 0: 
    print("Probability invalid!")
    return False
  if fullColumns[col1]:
    print("First column given is full!")
    return False
  if fullColumns[col2]:
    print("Second column given is full!")
    return False
  if not checkMeasure(col1):
    return False
  if not checkMeasure(col2):
    return False
  return True

In [1]:
def displayBoard():
    board = ['0'] * num_cols * num_rows

    for i in range(num_cols):
        for j in range(len(movesByColumn[i])):
            board[i + (num_rows - 1 - j) * num_cols] = movesByTime[movesByColumn[i][j]]

    for i in range(num_cols * num_rows):
        print(board[i] + " ", end="")
        if i % 5 == 4:
            print() #newline

In [ ]:
def checkDone():
  return sum(fullColumns) == NUM_COLUMN

In [31]:
def checkWin(row, col):
  assert(0 <= row < NUM_ROW)
  assert(0 <= col < NUM_COLUMN)
  # Check row for win
  x, y=row, row
  i = row + 1
  while i < NUM_ROW:
    if board[i][col] != board[row][col]:
      x = i - 1
      break
    i += 1
  if i == NUM_ROW:
    x = NUM_ROW - 1
  i = row - 1
  while i > -1:
    if board[i][col] != board[row][col]:
      y = i + 1
      break
    i -= 1
  if i == -1:
    y = 0
  if x - y + 1 >= 4:
    return True
  
  # Check col for win
  x, y = col, col
  j = col + 1
  while j < NUM_COLUMN:
    if board[row][j] != board[row][col]:
      x = j - 1
      break
    j += 1
  if j == NUM_COLUMN:
    x = j - 1
  j = col - 1
  while j > -1:
    if board[row][j] != board[row][col]:
      y = j + 1
      break
    j -= 1
  if j == -1:
    y = 0
  if x - y + 1 >= 4:
    return True

  # Check diagonal 1 for win
  x, y = 0, 0
  i = 1
  while row + i < NUM_ROW and col+i < NUM_COLUMN:
    if board[row + i][col + i] != board[row][col]:
      x = i - 1
      break
    i += 1
  if row + i == NUM_ROW or col + i == NUM_COLUMN:
    x = i - 1
  i = -1
  while row + i > -1 and col + i > -1:
    if board[row + i][col + i] != board[row][col]:
      x = i + 1
      break
    i -= 1
  if row + i == -1 or col + i == -1:
    y = i + 1
  if x - y + 1 >= 4:
    return True

  # Check diagonal 2 for win
  x, y = 0, 0
  i = 1
  while row + i < NUM_ROW and col - i > -1:
    if board[row + i][col - i] != board[row][col]:
      x = i - 1
      break
    i += 1
  if row + i == NUM_ROW or col - i == -1:
    x = i - 1
  i = -1
  while row + i > -1 and col - i < NUM_COLUMN:
    if board[row + i][col - i] != board[row][col]:
      x = i + 1
      break
    i -= 1
  if row + i == -1 or col - i == NUM_COLUMN:
    y = i + 1
  if x - y + 1 >= 4:
    return True

  return False

In [ ]:
def classicConnectFour():
  curInColumn = [0 for i in range(NUM_COLUMN)]
  winner = -1
  for t in range(len(movesByTime)):
    move = movesByTime[t]
    curRow = curInColumn[move.col1]
    board[move.col1][curRow]=move.player
    curInColumn[move.col1]+=1
    if checkWin(curRow, move.col1):
      -
      winner = move.player
      break
  # displayClassic()
  print("Player " + winner + " won! Congrats!")

In [ ]:
# Initial interaction with player(s)
print("Welcome to Connect 4 - Quantum Edition!")

displayBoard()
while not checkDone():
  getMove(curPlayer)
  curPlayer = 1-curPlayer
# Classic board
# NOTE: ELEMENTS ARE INDEXED BY [row][col] NOT THE OTHER WAY AROUND!
board = [[-1 for i in range(NUM_COLUMN)] for j in range(NUM_ROW)]
classicConnectFour()